In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from tdc.single_pred import ADME
from DeepPurpose import utils, CompoundPred
from tdc.single_pred import ADME
from sklearn.model_selection import cross_val_score, KFold, cross_val_predict
from sklearn import metrics
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

In [2]:
data = ADME(name = 'Solubility_AqSolDB').get_data(format = 'dict')
X, y = data['Drug'], data['Y']

Downloading...
100%|███████████████████████████████████████████████████████████████████████████████| 853k/853k [00:01<00:00, 720kiB/s]
Loading...
Done!


In [3]:
print(y.mean())
print(y.max())
print(y.min())
print(np.median(y))

-2.8899088047869865
2.1376816201
-13.1719
-2.61817341795


In [4]:
X_morgan = [AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(smiles), 2, nBits=1024) for smiles in X]
X_morgan_np = np.array([np.array(fp) for fp in X_morgan])
# 데이터 분할: 학습 및 테스트 세트
X_train, X_test, y_train, y_test = train_test_split(X_morgan_np, y, test_size=0.2, random_state=42)

# Cross-validation을 위한 fold 개수와 반복 횟수 설정
num_folds = 5
num_repeats = 10

# Cross-validation을 반복하여 각 반복에서의 점수를 저장할 리스트 생성
cv_mse_scores = []
cv_mape_scores = []
cv_mae_scores = []
cv_r2_scores = []

# KFold 객체를 생성하여 데이터를 섞고 나누는 인덱스 생성
kf = KFold(n_splits=num_folds, shuffle=True)

[12:37:41] WARNING: not removing hydrogen atom without neighbors
[12:37:41] WARNING: not removing hydrogen atom without neighbors
[12:37:41] WARNING: not removing hydrogen atom without neighbors
[12:37:41] WARNING: not removing hydrogen atom without neighbors
[12:37:41] WARNING: not removing hydrogen atom without neighbors
[12:37:41] WARNING: not removing hydrogen atom without neighbors
[12:37:41] WARNING: not removing hydrogen atom without neighbors
[12:37:41] WARNING: not removing hydrogen atom without neighbors
[12:37:41] WARNING: not removing hydrogen atom without neighbors
[12:37:41] WARNING: not removing hydrogen atom without neighbors
[12:37:41] WARNING: not removing hydrogen atom without neighbors
[12:37:41] WARNING: not removing hydrogen atom without neighbors
[12:37:41] WARNING: not removing hydrogen atom without neighbors
[12:37:41] WARNING: not removing hydrogen atom without neighbors
[12:37:41] WARNING: not removing hydrogen atom without neighbors
[12:37:41] WARNING: not r

In [5]:
# 반복적으로 cross-validation 수행
for i in range(num_repeats):
    # 모델 정의
    model = RandomForestRegressor(min_samples_leaf=4, min_samples_split=4 ,n_estimators=100, random_state=42)
    
    mse_scores = -cross_val_score(model, X_train, y_train, cv=kf, scoring='neg_mean_squared_error')
    mae_scores = -cross_val_score(model, X_train, y_train, cv=kf, scoring='neg_mean_absolute_error')
    mape_scores = -cross_val_score(model, X_train, y_train, cv=kf, scoring='neg_mean_absolute_percentage_error')
    r2_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='r2')

    cv_mse_scores.append(mse_scores.mean())
    cv_mape_scores.append(mape_scores.mean())
    cv_mae_scores.append(mae_scores.mean())
    cv_r2_scores.append(r2_scores.mean())


In [6]:
# Cross-validation 점수의 평균 계산
mean_mse_score = np.mean(cv_mse_scores)
mean_mae_score = np.mean(cv_mae_scores)
mean_mape_score = np.mean(cv_mape_scores)
mean_r2_score = np.mean(cv_r2_scores)

# Cross-validation 점수의 표준 편차 계산
std_dev_mse = np.std(cv_mse_scores)
std_dev_mae = np.std(cv_mae_scores)
std_dev_mape = np.std(cv_mape_scores)
std_dev_r2 = np.std(cv_r2_scores)

# 평균과 표준 편차 출력
print("평균 Cross-validation MSE:", mean_mse_score)
print("Cross-validation MSE 표준 편차:", std_dev_mse)

print("평균 Cross-validation MAE:", mean_mae_score)
print("Cross-validation MAE 표준 편차:", std_dev_mae)

print("평균 Cross-validation MAPE:", mean_mape_score)
print("Cross-validation MAPE 표준 편차:", std_dev_mape)

print("평균 Cross-validation R square:", mean_r2_score)
print("Cross-validation R square 표준 편차:", std_dev_r2)

평균 Cross-validation MSE: 2.125849233491258
Cross-validation MSE 표준 편차: 0.00899325442291375
평균 Cross-validation MAE: 1.070882572529678
Cross-validation MAE 표준 편차: 0.005201967023674141
평균 Cross-validation MAPE: 1470681346503.5996
Cross-validation MAPE 표준 편차: 113781949948.9357
평균 Cross-validation R square: 0.6253337136841204
Cross-validation R square 표준 편차: 0.0027997298867052187
